In [ ]:
!pip install opencv-python imgbeddings psycopg2-binary

In [ ]:
# importing the cv2 library
import cv2

# loading the haar case algorithm file into alg variable
alg = "/content/haarcascade_frontalface_default (2).xml"
# passing the algorithm to OpenCV
haar_cascade = cv2.CascadeClassifier(alg)
# loading the image path into file_name variable - replace <INSERT YOUR IMAGE NAME HERE> with the path to your image
file_name = "<INSERT YOUR IMAGE NAME HERE>"
# reading the image
img = cv2.imread(file_name, 0)
# creating a black and white version of the image
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
# detecting the faces
faces = haar_cascade.detectMultiScale(
    gray_img, scaleFactor=1.05, minNeighbors=5, minSize=(100, 100)
)

i = 0
# for each face detected
for x, y, w, h in faces:
    # crop the image to select only the face
    cropped_image = img[y : y + h, x : x + w]
    # loading the target image path into target_file_name variable  - replace <INSERT YOUR TARGET IMAGE NAME HERE> with the path to your target image
    target_file_name = 'stored-faces/' + str(i) + '.jpg'
    cv2.imwrite(
        target_file_name,
        cropped_image,
    )
    i = i + 1;

In [ ]:
# Install required libraries
!pip install torch torchvision torchaudio
!pip install imgbeddings
!pip install Pillow
!pip install psycopg2


In [ ]:
# importing the required libraries
import numpy as np
from imgbeddings import imgbeddings
from PIL import Image
import psycopg2
import os

# connecting to the database - replace the SERVICE URI with the service URI
conn = psycopg2.connect("SERVICE URI")

for filename in os.listdir("stored-faces"):
    # opening the image
    img = Image.open("stored-faces/" + filename)
    # loading the `imgbeddings`
    ibed = imgbeddings()
    # calculating the embeddings
    embedding = ibed.to_embeddings(img)
    cur = conn.cursor()
    # Check if the record exists before attempting to insert
    cur.execute("SELECT * FROM pictures WHERE picture = %s", (filename,))
    existing_record = cur.fetchone()

    if existing_record:
      print(f"Record for {filename} already exists, skipping insertion.")
    else:
      try:
        cur.execute("INSERT INTO pictures (picture, embedding) VALUES (%s, %s)", (filename, embedding[0].tolist()))
        print(f"Inserted record for {filename}.")
      except psycopg2.IntegrityError as e:
        conn.rollback()
        print(f"Error inserting {filename}: {e}")
      else:
        conn.commit()



    print(filename)
conn.commit()

In [ ]:
# Ensure necessary packages are installed
!pip install torch torchvision torchaudio
!pip install imgbeddings
!pip install Pillow
!pip install psycopg2-binary

# Import libraries
import numpy as np
from imgbeddings import imgbeddings
from PIL import Image
import psycopg2

# Verify import of PIL.Image
print(f"PIL Image module: {Image}")

# Load the face image path into file_name variable
file_name = " path to your image"  # replace with the path to your image

# Open the image
try:
    img = Image.open(file_name)
    print("Image opened successfully.")
except Exception as e:
    print(f"Error opening image: {e}")

# Load the `imgbeddings`
try:
    ibed = imgbeddings()
    print("Imgeddings loaded successfully.")
except Exception as e:
    print(f"Error loading imgbeddings: {e}")

# Calculate the embeddings
try:
    embedding = ibed.to_embeddings(img)
    print("Embeddings calculated successfully.")
    print(embedding)
except Exception as e:
    print(f"Error calculating embeddings: {e}")


PIL Image module: <module 'PIL.Image' from '/usr/local/lib/python3.10/dist-packages/PIL/Image.py'>
Image opened successfully.
Imgeddings loaded successfully.
Embeddings calculated successfully.
[[ 3.16207200e-01  6.65520012e-01 -2.05005690e-01 -1.00516355e+00
   1.01549938e-01 -9.31174159e-01  2.14451551e-01  3.84461880e-01
   6.17327929e-01 -2.15764567e-01  6.99230075e-01  2.63415545e-01
   1.06047325e-01 -6.23790808e-02  9.72768188e-01  7.86838353e-01
   5.90120137e-01 -4.04453516e-01 -3.97240818e-02 -4.91379499e-01
   3.41800898e-01  3.63723904e-01 -8.09900701e-01 -6.16111159e-02
  -1.90738887e-01 -7.25090876e-02  2.12913603e-01  2.71911591e-01
   3.40515934e-03  2.00910711e+00  1.00681469e-01  1.36178374e-01
   1.40502974e-01 -1.33898616e-01  8.83627951e-01 -1.70629472e-01
   1.76870711e-02  5.03487825e-01  7.15238154e-01  1.15967151e-02
   1.14352679e+00  3.90566826e-01  5.34125209e-01 -3.90546709e-01
  -3.92514408e-01 -4.70361620e-01 -4.23651397e-01  7.52506912e-01
   1.60583526e

In [ ]:
# loading the face image path into file_name variable
file_name = "<INSERT YOUR FACE FILE NAME>"  # replace <INSERT YOUR FACE FILE NAME> with the path to your image
# opening the image
img = Image.open(file_name)
# loading the `imgbeddings`
ibed = imgbeddings()
# calculating the embeddings
embedding = ibed.to_embeddings(img)

In [ ]:
from IPython.display import Image, display

cur = conn.cursor()
string_representation = "["+ ",".join(str(x) for x in embedding[0].tolist()) +"]"
cur.execute("SELECT * FROM pictures ORDER BY embedding <-> %s LIMIT 1;", (string_representation,))
rows = cur.fetchall()
for row in rows:
    display(Image(filename="stored-faces/"+row[0]))
cur.close()